In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split

In [1]:

class FashionMNISTModel(nn.Module):
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load dataset with transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

# Split into Train (80%), Validation (10%), Test (10%)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

100%|██████████| 26.4M/26.4M [00:01<00:00, 14.0MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 319kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 4.92MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 5.10MB/s]


In [2]:

# Setup model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FashionMNISTModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [3]:

# Training loop with validation metrics
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    correct_train, total_train, train_loss = 0, 0, 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        train_loss += loss.item()

    train_acc = 100 * correct_train / total_train
    avg_train_loss = train_loss / len(train_loader)

    # Validation Phase
    model.eval()
    correct_val, total_val, val_loss = 0, 0, 0.0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            val_loss += loss.item()

    val_acc = 100 * correct_val / total_val
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.2f}%")

Epoch 1/5: Train Loss: 0.4698, Train Acc: 83.03% | Val Loss: 0.3352, Val Acc: 88.05%
Epoch 2/5: Train Loss: 0.3022, Train Acc: 89.01% | Val Loss: 0.2803, Val Acc: 90.20%
Epoch 3/5: Train Loss: 0.2517, Train Acc: 90.70% | Val Loss: 0.2604, Val Acc: 90.93%
Epoch 4/5: Train Loss: 0.2205, Train Acc: 91.94% | Val Loss: 0.2453, Val Acc: 91.23%
Epoch 5/5: Train Loss: 0.1915, Train Acc: 92.92% | Val Loss: 0.2363, Val Acc: 91.60%


In [7]:
from sklearn.metrics import classification_report

# Test Evaluation
model.eval()
correct_test, total_test = 0, 0
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

        # Store predictions and labels for classification report
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = 100 * correct_test / total_test
print(f"Final Test Accuracy: {test_acc:.2f}%")

# Generate Classification Report
class_names = dataset.classes  # Get class names from FashionMNIST dataset
print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds, target_names=class_names))

Final Test Accuracy: 91.57%

Classification Report:

              precision    recall  f1-score   support

 T-shirt/top       0.90      0.83      0.86       610
     Trouser       0.98      0.99      0.99       592
    Pullover       0.85      0.88      0.87       583
       Dress       0.93      0.92      0.93       621
        Coat       0.86      0.86      0.86       627
      Sandal       0.96      0.99      0.98       596
       Shirt       0.76      0.77      0.77       591
     Sneaker       0.96      0.96      0.96       572
         Bag       0.98      0.99      0.99       594
  Ankle boot       0.98      0.96      0.97       614

    accuracy                           0.92      6000
   macro avg       0.92      0.92      0.92      6000
weighted avg       0.92      0.92      0.92      6000



In [6]:
# Save trained model
torch.save(model, "fashion_mnist_full_model.pth")
print("Model saved!")


Model saved!
